# Assignment 4: Build a BERT-based Sentiment Classification (100 Points)

Instructor: Ziyu Yao; Class: CS478 Fall 2024

This assignment will extend from the last one. The goal is to build a binary sentiment classifier based on BERT. We will use Hugging Face transformers library (https://huggingface.co/docs/transformers/index). The assignment has two parts:
- **Part 1 (70 points):** Evaluate a public, BERT-based sentiment classifier on our movie review sentiment dataset. In this part, no fine-tuning is needed.
- **Part 2 (30 points):** Further fine-tune the pre-trained classifier on our sentiment classification dataset, hoping that it will give us a better accuracy on our movie review dataset.

Since we will be fine-tuning a BERT model, you are suggested to use Google Colab and set up a GPU card (free resource). However, the assignment can be completed locally as well, although the training will take more time.

To get started, please make sure to install the transformers library:

In [2]:
pip install -U jupyter ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 139.8/139.8 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 214.4/214.4 kB ? eta 0:00:00
   ---------------------------------------- 2.3/2.3 MB 49.5 MB/s eta 0:00:00
   ---------------------------------------- 8.9/8.9 MB 51.8 MB/s eta 0:00:00
   --------------------------------------- 529.8/529.8 kB 32.5 MB/s eta 0:00:00
   --------------------------------------- 290.4/290.4 kB 17.5 MB/s eta 0:00:00
   --------------------------------------- 162.5/162.5 kB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 133.3/133.3 kB ? eta 0:00:00
   --------------------------------------- 347.5/347.5 kB 22.5 MB/s eta 0:00:00
   ---------------------------------------- 57.3/57.3 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 48.0/48.0 kB ? eta 0:00:00
   ---------------------------------------- 10.0/10.0 MB 53.4 MB/s eta 0:

In [4]:
import torch

from typing import List, Dict
import random
import numpy as np
from collections import Counter
import os

# transformers
import transformers

# Set up overall seed
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

## Part 0: Data Structure and Loading

This part is adapted from the previous assignment, including reusing the data structure and evaluation scripts that we have previously defined (but with modifications). _There's nothing to fill out in this part, but please study the code and particularly check the modifications that we made to accommodate the BERT classifier._

First, we will define the data structure for storing our sentiment data. Compared with the previous version, we have removed the naive sentence tokenization based on whitespace split. Later on, we will use the tokenizer of the pre-trained model for sentence tokenization.

In [6]:
class SentimentExample:
    """
    Data wrapper for a single example for sentiment analysis.

    Attributes:
        sentence (string): a string-type sentence (untokenized)
        label (int): 0 or 1 (0 = negative, 1 = positive)
        word_indices (List[int]): list of word indices in the vocab, which will generated by the `indexing_sentiment_examples` method
    """

    def __init__(self, sentence, label):
        self.sentence = sentence
        self.label = label
        self.words = None
        self.word_indices = None

    def __repr__(self):
        return self.sentence + "; label=" + repr(self.label)

    def __str__(self):
        return self.__repr__()

def read_sentiment_examples(infile: str) -> List[SentimentExample]:
    """
    Reads sentiment examples in the format [0 or 1]<TAB>[raw sentence]; tokenizes and cleans the sentences and forms
    SentimentExamples. Note that all words have been lowercased.

    :param infile: file to read from
    :return: a list of SentimentExamples parsed from the file
    """
    f = open(infile)
    exs = []
    for line in f:
        if len(line.strip()) > 0:
            line = line.strip()
            fields = line.split("\t")
            if len(fields) != 2:
                fields = line.split()
                label = 0 if "0" in fields[0] else 1
                sent = " ".join(fields[1:])
            else:
                # Slightly more robust to reading bad output than int(fields[0])
                label = 0 if "0" in fields[0] else 1
                sent = fields[1]
            sent = sent.lower() # lowercasing
            exs.append(SentimentExample(sent, label))
    f.close()
    return exs


def read_blind_sst_examples(infile: str) -> List[SentimentExample]:
    """
    Reads the blind SST test set, which just consists of unlabeled sentences. Note that all words have been lowercased.
    :param infile: path to the file to read
    :return: list of tokenized sentences (list of list of strings)
    """
    f = open(infile, encoding='utf-8')
    exs = []
    for line in f:
        if len(line.strip()) > 0:
            line = line.strip()
            sent = line.lower()
            exs.append(SentimentExample(sent, label=-1)) # pseudo label -1
    return exs


def write_sentiment_examples(exs: List[SentimentExample], outfile: str):
    """
    Writes sentiment examples to an output file with one example per line, the predicted label followed by the example.
    Note that what gets written out is tokenized.
    :param exs: the list of SentimentExamples to write
    :param outfile: out path
    :return: None
    """
    o = open(outfile, 'w')
    for ex in exs:
        o.write(repr(ex.label) + "\t" + ex.sentence + "\n")
    o.close()

Now, load the training, dev, and test sets (the same sets as in the prior assignment):

(Note: The following commands is for Google Colab users only. If you run the notebook locally, change the path configuration accordingly.)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# Specify the data paths
train_path = "H:\\Projects\\CS478\\Assignment3\\data\\train.txt"
dev_path = "H:\\Projects\\CS478\\Assignment3\\data\\dev.txt"
blind_test_path = "H:\\Projects\\CS478\\Assignment3\\data\\test-blind.txt" # blind test

# Load train, dev, and test exs and index the words.
train_exs = read_sentiment_examples(train_path)
dev_exs = read_sentiment_examples(dev_path)
test_exs_words_only = read_blind_sst_examples(blind_test_path)
print(repr(len(train_exs)) + " / " + repr(len(dev_exs)) + " / " + repr(len(test_exs_words_only)) + " train/dev/test examples")

6920 / 872 / 1821 train/dev/test examples


Next, we set up the evaluation metrics for sentiment classification. The function is the same as before.

In [8]:
def calculate_metrics(golds: List[int], predictions: List[int], print_only: bool=False):
    """
    Calculate evaluation statistics comparing golds and predictions, each of which is a sequence of 0/1 labels.
    Returns accuracy, precision, recall, and F1.

    :param golds: gold labels
    :param predictions: pred labels
    :param print_only: set to True if printing the stats without returns
    :return: accuracy, precision, recall, and F1 (all floating numbers), or None (when print_only is True)
    """
    num_correct = 0
    num_pos_correct = 0
    num_pred = 0
    num_gold = 0
    num_total = 0
    if len(golds) != len(predictions):
        raise Exception("Mismatched gold/pred lengths: %i / %i" % (len(golds), len(predictions)))
    for idx in range(0, len(golds)):
        gold = golds[idx]
        prediction = predictions[idx]
        if prediction == gold:
            num_correct += 1
        if prediction == 1:
            num_pred += 1
        if gold == 1:
            num_gold += 1
        if prediction == 1 and gold == 1:
            num_pos_correct += 1
        num_total += 1
    acc = float(num_correct) / num_total
    prec = float(num_pos_correct) / num_pred if num_pred > 0 else 0.0
    rec = float(num_pos_correct) / num_gold if num_gold > 0 else 0.0
    f1 = 2 * prec * rec / (prec + rec) if prec > 0 and rec > 0 else 0.0

    print("Accuracy: %i / %i = %f" % (num_correct, num_total, acc))
    print("Precision (fraction of predicted positives that are correct): %i / %i = %f" % (num_pos_correct, num_pred, prec)
          + "; Recall (fraction of true positives predicted correctly): %i / %i = %f" % (num_pos_correct, num_gold, rec)
          + "; F1 (harmonic mean of precision and recall): %f" % f1)

    if not print_only:
        return acc, prec, rec, f1

## Part 1: Evaluate a Pre-trained Sentiment Classifier (70 Points)

In this part, we will evaluate a pre-trained sentiment classifier ("textattack/bert-base-uncased-yelp-polarity") on our sentiment classification dataset. This classifier, as shown by its name, was pre-trained on BERT (uncased base version) and then fine-tuned on a Yelp polarity dataset. It has two class labels, 0 representing negative and 1 representing positive. The model is open-sourced on the tranformers model hub (https://huggingface.co/textattack/bert-base-uncased-yelp-polarity). We will test how well the model performs on the dev set, without fine-tuning.

To this end, we will do three things:
- Create and load a tokenizer for the pre-trained model, using `BertTokenizer` (https://huggingface.co/docs/transformers/v4.25.1/en/model_doc/bert#transformers.BertTokenizer);
- Create and load the pre-trained classifier, using `BertForSequenceClassification` (https://huggingface.co/docs/transformers/v4.25.1/en/model_doc/bert#transformers.BertForSequenceClassification);
- Run this model on our dev set and report its accuracy.

Please follow the instructions and fill out the missing code.

**Recommended Material:** You may find the example code of `BertForSequenceClassification` helpful: https://huggingface.co/docs/transformers/v4.25.1/en/model_doc/bert#transformers.BertForSequenceClassification.forward.example.

First, let's load the required libraries:

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification

Define the pre-trained checkpoint name:

In [11]:
pretrained_checkpoint = "textattack/bert-base-uncased-yelp-polarity"

Follow the example here (https://huggingface.co/docs/transformers/v4.25.1/en/model_doc/bert#transformers.BertForSequenceClassification.forward.example), create a tokenizer and load the pre-trained model of `textattack/bert-base-uncased-yelp-polarity`.

<font color='blue'>YOUR TASK: Complete the following code for creating the tokenizer and the pre-trained model.</font>

In [12]:
# TODO
tokenizer = BertTokenizer.from_pretrained(pretrained_checkpoint)
model = BertForSequenceClassification.from_pretrained(pretrained_checkpoint)

C:\Users\malma\AppData\Roaming\Python\Python37\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\malma\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


As before, we can "print" the model and see its structure:

In [13]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

As before, we will project the model to the device:

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cpu


Now, let's see how to use this classifier. Consider the first sentence in our dev set:

In [15]:
dev_exs[0]

it 's a lovely film with lovely performances by buy and accorsi .; label=1

Tokenize the sentence using the created tokenizer:

<font color='blue'>YOUR TASK: Complete the following code for tokenizing the first sentence in the dev set.</font>

In [28]:
# TODO
inputs = tokenizer(dev_exs[0].sentence, return_tensors='pt', padding=True, truncation=True)

See what it contains:

In [29]:
inputs

{'input_ids': tensor([[  101,  2009,  1005,  1055,  1037,  8403,  2143,  2007,  8403,  4616,
          2011,  4965,  1998, 16222,  5668,  2072,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

The tokenization results contain three pieces:
- `input_ids` of shape `(batch_size x batch_max_length)`, which is a tensor of the tokenized and indexed input sentences. Since there's only one sentence to the tokenizer, the first dimension (`batch_size`) is 1 and the second dimension is the length of the input sentence.
- `token_type_ids` of shape `(batch_size x batch_max_length)`, which is an all-zero tensor because there's only one "type" of input (i.e., a single task input sentence).
- `attention_mask` of shape `(batch_size x batch_max_length)`, with 1 indicating valid attention and 0 otherwise. Here the tensor is all ones because all words are valid for being attended.

You should also see `device='cuda:0'` if you are using GPU.

Now, run the model inference given this sentence. You should use the "inputs" defined above. The returned "outputs" should contain the model logits.

<font color='blue'>YOUR TASK: Complete the following code for classifying the first sentence in the dev set.</font>

Hint: if you receive an error "Expected all tensors to be on the same device", please apply `.to(device)` to your `inputs`.

In [31]:
# TODO:
outputs = model(**inputs)

See what it contains:

In [32]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-4.5182,  4.2525]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

The classifier output has multiple items. Among them:
- `logits` is the logits (i.e., values before softmax) from the classifier.
- `loss` is the model (cross entropy) loss on the given input, which is now `None` because there's no ground-truth labels provided.

Calculate the prediction labels (1 for positive and 0 for negative) based on the model logits:

<font color='blue'>YOUR TASK: Complete the following code for getting the prediction label.</font>

In [33]:
# TODO:
predicted_class_id = (torch.argmax(outputs.logits, dim=1)).item()
print("Prediction:", predicted_class_id)

Prediction: 1


While the above code can process a single input sentence, to make the best use of the GPU, we want to batchify this process, including tokenizing a batch of input sentences, and running the loaded classifier to make predictions for the batch data.

<font color='blue'>YOUR TASK: Complete the `TODO` to implement `SentimentExampleBatchIterator`, which loads and tokenizes a batch of input data at a time.</font>

**Hint:** You would need to specify the `padding` and `truncation` options for tokenizing a batch of data.

In [44]:
class SentimentExampleBatchIterator:
    """
    A batch iterator which will produce the next batch indexed data.

    Attributes:
        data: a list of SentimentExample objects, which is the source data input
        batch_size: an integer number indicating the number of examples in each batch
        PAD_idx: the index of PAD in the vocabulary
        shuffle: whether to shuffle the data (should set to True only for training)
    """
    def __init__(self, data: List[SentimentExample], batch_size: int, shuffle: bool=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle

        self._indices = None
        self._cur_idx = None

    def refresh(self):
        self._indices = list(range(len(self.data)))
        if self.shuffle:
            random.shuffle(self._indices)
        self._cur_idx = 0

    def get_next_batch(self, tokenizer): # note that we now feed the tokenizer as an argument, which decides how a sentence needs to be tokenized
        if self._cur_idx < len(self.data): # loop over the dataset
            st_idx = self._cur_idx
            if self._cur_idx + self.batch_size > len(self.data) - 1:
                ed_idx = len(self.data)
            else:
                ed_idx = self._cur_idx + self.batch_size
            self._cur_idx = ed_idx # update
            # Retrieve a batch of SentimentExample data. Note that the data is NOT yet tokenized.
            batch_exs = [self.data[self._indices[_idx]] for _idx in range(st_idx, ed_idx)]

            # TODO: tokenize the `batch_exs` data. The resulting `batch_inputs` will have
            # exactly the same fields as `inputs`, i.e., input_ids, token_type_ids, and attention_mask.
            # However, you should see that now each field contains the result from a batch of data with paddings.
            # Hint: https://huggingface.co/course/chapter2/2?fw=pt#preprocessing-with-a-tokenizer
            batch_inputs = tokenizer([ex.sentence for ex in batch_exs], return_tensors='pt', padding=True, truncation=True)

            # collect the human labels in this batch
            batch_labels = torch.tensor(np.array([ex.label for ex in batch_exs], dtype=np.int64)).to(device)
            return batch_inputs, batch_labels
        else:
            return None

<font color='blue'>YOUR TASK: Complete the `TODO` to make prediction on a batch of data each time, using the loaded pretrained classifier (the `classifier` argument).</font>

In [45]:
def batch_predict(classifier, batch_inputs: torch.Tensor) -> List[int]:
    # TODO: implement the batch prediction using the `classifier` (the loaded BERT-based model)
    # The function should return a list of 1/0 predicted labels
    with torch.no_grad():
        outputs = classifier(**batch_inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).tolist()
    return preds

def evaluate(classifier, exs: List[SentimentExample], return_metrics: bool=False):
    """
    Evaluates a given classifier on the given examples
    :param classifier: classifier to evaluate
    :param exs: the list of SentimentExamples to evaluate on
    :param return_metrics: set to True if returning the stats
    :return: None (but prints output)
    """
    all_labels = []
    all_preds = []

    eval_batch_iterator = SentimentExampleBatchIterator(exs, batch_size=32, shuffle=False) # hard-coded batch size
    eval_batch_iterator.refresh()
    batch_data = eval_batch_iterator.get_next_batch(tokenizer)
    while batch_data is not None:
        batch_inputs, batch_labels = batch_data

        all_labels += list(batch_labels)

        preds = batch_predict(classifier, batch_inputs)
        all_preds += preds

        batch_data = eval_batch_iterator.get_next_batch(tokenizer)

    if return_metrics:
        acc, prec, rec, f1 = calculate_metrics(all_labels, all_preds)
        return acc, prec, rec, f1
    else:
        calculate_metrics(all_labels, all_preds, print_only=True)

Now, let's run the model and evaluate its performance on the dev set!

(If your implementation is correct, you should see an accuracy of around 0.8.)

In [46]:
evaluate(model, dev_exs)

Accuracy: 686 / 872 = 0.786697
Precision (fraction of predicted positives that are correct): 357 / 456 = 0.782895; Recall (fraction of true positives predicted correctly): 357 / 444 = 0.804054; F1 (harmonic mean of precision and recall): 0.793333


## Part 2: Fine-tune a BERT-based Model for Sentiment Classification (30 Points)

While in the last part we directly load in a sentiment classifier and use it in our movie review sentiment classification task, in this part, we will fine-tune this classifier on our dataset and see if it can achieve a better task performance in the end.

Like before, we will create an optimizer:

In [47]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),
  lr = 5e-5, eps = 1e-8)

<font color='blue'>YOUR TASK: Complete the `TODO` for model training.</font>

In [48]:
import time

BATCH_SIZE = 32
N_EPOCHS = 5

# create a batch iterator for the training data
batch_iterator = SentimentExampleBatchIterator(train_exs, batch_size=BATCH_SIZE, shuffle=True)

# training
best_epoch = -1
best_acc = -1
start_time = time.time()
for epoch in range(N_EPOCHS):
    print("Epoch %i" % epoch)

    batch_iterator.refresh() # initiate a new iterator for this epoch

    model.train() # turn on the "training mode"
    batch_loss = 0.0
    batch_example_count = 0
    batch_data = batch_iterator.get_next_batch(tokenizer)
    while batch_data is not None:
        batch_inputs, batch_labels = batch_data
        model.zero_grad()

        # TODO: call the model and compute the loss
        # Hint: you don't need to implement the loss on your own when using the transformers `BertForSequenceClassification`;
        # see https://huggingface.co/docs/transformers/v4.25.1/en/model_doc/bert#transformers.BertForSequenceClassification.forward.example for help
        batch_outputs = model(**batch_inputs, labels=batch_labels)
        loss = batch_outputs.loss

        # record the loss and number of examples, so we could report some stats
        batch_example_count += len(batch_labels)
        batch_loss += loss.item() * len(batch_labels)

        # backpropagate the loss
        loss.backward()
        optimizer.step()

        # get another batch
        batch_data = batch_iterator.get_next_batch(tokenizer)

    print("Avg loss: %.5f" % (batch_loss / batch_example_count))

    # evaluate on dev set
    model.eval() # turn on the "evaluation mode"
    acc, _, _, _ = evaluate(model, dev_exs, return_metrics=True)
    if acc > best_acc:
        best_acc = acc
        best_epoch = epoch
        print("Secure a new best accuracy %.3f in epoch %d!" % (best_acc, best_epoch))

        # save the current best model parameters
        print("Save the best model checkpoint as `best_model.ckpt`!")
        # torch.save(model.state_dict(), "best_model.ckpt")
        model.save_pretrained("best_bert_model.ckpt")

    print("Time elapsed: %s" % time.strftime("%Hh%Mm%Ss", time.gmtime(time.time()-start_time)))
    print("-" * 10)

# load back the best checkpoint on dev set
print("End of training! The best accuracy %.3f was obtained in epoch %d." % (best_acc, best_epoch))
# model.load_state_dict(torch.load("best_model.ckpt"))
model = BertForSequenceClassification.from_pretrained("best_bert_model.ckpt").to(device)

Epoch 0
Avg loss: 0.37332
Accuracy: 764 / 872 = 0.876147
Precision (fraction of predicted positives that are correct): 376 / 416 = 0.903846; Recall (fraction of true positives predicted correctly): 376 / 444 = 0.846847; F1 (harmonic mean of precision and recall): 0.874419
Secure a new best accuracy 0.876 in epoch 0!
Save the best model checkpoint as `best_model.ckpt`!
Time elapsed: 00h22m06s
----------
Epoch 1
Avg loss: 0.13756
Accuracy: 753 / 872 = 0.863532
Precision (fraction of predicted positives that are correct): 364 / 403 = 0.903226; Recall (fraction of true positives predicted correctly): 364 / 444 = 0.819820; F1 (harmonic mean of precision and recall): 0.859504
Time elapsed: 00h44m05s
----------
Epoch 2
Avg loss: 0.04053
Accuracy: 772 / 872 = 0.885321
Precision (fraction of predicted positives that are correct): 386 / 428 = 0.901869; Recall (fraction of true positives predicted correctly): 386 / 444 = 0.869369; F1 (harmonic mean of precision and recall): 0.885321
Secure a new 

By estimation, you should be able to complete the model training in 10 minutes (or significantly faster, depending on whether you used GPU or not; for my record, I finished training in 6 minutes when using the T4 GPU card in Colab.)

Report the best performance of your model and the epoch you obtained it.
How does the fine-tuned BERT model compare with (a) its counterpart without fine-tuning and (b) the feedforward neural network which we trained in the previous assignment? Why do you think the fine-tuned BERT could have the observed performance?

<font color='blue'>YOUR TASK: provide your answer in the LaTex PDF.</font>

The following code will evaluate your model on the blind test set, with results saved in the data_to_submit folder. 

<font color='blue'>YOUR TASK: Submit your test output (i.e., your predictions) to Blackboard.</font>

In [51]:
all_preds = [] # save the prediction results

# iterator to load the test set
eval_batch_iterator = SentimentExampleBatchIterator(test_exs_words_only, batch_size=32, shuffle=False)
eval_batch_iterator.refresh()
batch_data = eval_batch_iterator.get_next_batch(tokenizer)
while batch_data is not None:
    batch_inputs, _ = batch_data
    preds = batch_predict(model, batch_inputs) # the `preds` shoud be a list of prediction labels
    all_preds += preds # accumulate the labels
    batch_data = eval_batch_iterator.get_next_batch(tokenizer)

# write the predicted labels along with its original sentence
test_output_path = "H:\\Projects\\CS478\\Assignment4\\test-blind.bert-output.txt"
test_exs_predicted = [SentimentExample(ex.sentence, all_preds[ex_idx]) for ex_idx, ex in enumerate(test_exs_words_only)]
write_sentiment_examples(test_exs_predicted, test_output_path)

## You have completed this assignment! Please upload your notebook and the test output file in `data_to_submit` to Blackboard. Complete the LaTex file with a shareable link to this notebook and upload the PDF to Gradescope.

**Note: please do NOT upload any model checkpoint as it can be too large to be taken by the system.**